In [1]:
import glob
import shutil
import sqlite3

import altair as alt
import numpy as np
import pandas as pd

# alt.data_transformers.disable_max_rows()
alt.data_transformers.enable("json")

DataTransformerRegistry.enable('json')

In [2]:
# Connect to database
dbname = "kalk_data.db"
eng = sqlite3.connect(dbname, detect_types=sqlite3.PARSE_DECLTYPES)

In [3]:
# Read tables
stn_df = pd.read_sql("SELECT * FROM stations", eng)
par_df = pd.read_sql("SELECT * FROM parameters_units", eng)
wc_df = pd.read_sql("SELECT * FROM water_chemistry", eng)
wc_df["sample_date"] = pd.to_datetime(wc_df["sample_date"], format="%Y-%m-%d %H:%M:%S")
wc_df["parameter_unit"] = wc_df["parameter"] + "_" + wc_df["unit"]

ax_scale = "Linear"

In [4]:
# Build drop-down list
par_list = ['None'] + sorted(wc_df["parameter_unit"].unique())
input_dropdown = alt.binding_select(options=par_list)
selection = alt.selection_single(
    fields=["parameter_unit"], bind=input_dropdown, name="Select"
)

In [5]:
# Ticks
ticks = alt.Chart(wc_df, height=150, width=450, title="Strip plot",
).add_selection(
    selection
).transform_filter(
    selection
).mark_tick(thickness=2, size=30, opacity=0.3,
).encode(
    x=alt.X("value:Q", title="Value", scale=alt.Scale(type=ax_scale.lower())),
    y=alt.Y(
        "lab:N",
        title="",
        sort=[
            "NIVA (historic)",
            "VestfoldLAB (historic)",
            "VestfoldLAB (2020)",
        ],
        ),
        color="lab:N",
        tooltip=[
            "vannmiljo_code:N",
            "sample_date:T",
            "lab:N",
            "parameter:N",
            "unit:N",
            "value:Q",
        ],
).interactive()

# Q-Q plot
base = alt.Chart(wc_df, height=300, width=450, title="Q-Q plot")

scatter = base.transform_filter(
    selection
).transform_quantile(
    'value',
    step=0.05,
    as_=['percentile', 'value'],
    groupby=['period'],
).transform_pivot(
    'period',
    groupby=['percentile'],
    value='value'
).mark_point().encode(
    x=alt.X('historic:Q', title="Historic data", scale=alt.Scale(type=ax_scale.lower())),
    y=alt.Y('new:Q', title="New data", scale=alt.Scale(type=ax_scale.lower())),
    color=alt.Color("percentile:Q", scale=alt.Scale(scheme="turbo")),
    tooltip=["percentile:Q", "historic:Q", "new:Q"],
).interactive()

# 1:1 line
line = base.transform_filter(
    selection
).transform_quantile(
    'value',
    step=0.05,
    as_=['percentile', 'value'],
    groupby=['period'],
).transform_pivot(
    'period',
    groupby=['percentile'],
    value='value'
).mark_line().encode(
    x=alt.X('historic:Q', title="", scale=alt.Scale(type=ax_scale.lower())),
    y=alt.Y('historic:Q', title="", scale=alt.Scale(type=ax_scale.lower())),
)

# KDE plot
kde = alt.Chart(wc_df, height=160, width=450, title="Density plot",
).transform_filter(
    selection
).transform_density(
    density="value",
    groupby=["lab"],
).mark_area(opacity=0.3,
).encode(
    x=alt.X("value:Q", title="Value", scale=alt.Scale(type=ax_scale.lower())),
    y=alt.Y("density:Q", title=""),
    color="lab:N",
    row=alt.Row(
        "lab:N",
        title="",
        sort=[
            "NIVA (historic)",
            "VestfoldLAB (historic)",
            "VestfoldLAB (2020)",
        ],
    ),
).interactive()

chart = (ticks & (scatter + line)) | kde
chart.configure_axis(labelFontSize=16, titleFontSize=20,
).configure_legend(labelFontSize=16)

chart.save('distribution_plots.json')

In [6]:
# Move files to 'pages' folder
flist = glob.glob("*.json")
for fpath in flist:
    shutil.move(fpath, "../pages/")

Error: Destination path '../pages/altair-data-8e3776c21c950fbcc70e890d74624289.json' already exists